In [3]:
import os

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [11]:
import sys
print(sys.path)

['/media/sanslab/Data/stevehoang/vinlp/evaluation', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/media/sanslab/Data/stevehoang/vinlp/venv/lib/python3.10/site-packages']


In [20]:
# !python -m pip install PyMuPDF

  Using cached PyMuPDFb-1.23.22-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.0 MB/s eta 0:00:00a 0:00:01
Using cached PyMuPDFb-1.23.22-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (30.6 MB)


In [22]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text()
    except Exception as e:
        print(f"Error: {e}")
    return text

pdf_path = "../data/Tu_duy_phan_bien.pdf"
text = extract_text_from_pdf(pdf_path)
print(text)

GIỚI THIỆU
Cuốn sách này không viết về những suy nghĩ của riêng tôi,
nhưng tôi được thôi thúc bởi những suy nghĩ của người khác.
Cuốn sách cũng không phải là tập hợp những bài thực hành tốt
nhất của tôi, nhưng tôi mô phỏng cách những người giỏi nhất
làm để trở nên tốt hơn.
Có nhiều khi suy nghĩ của chúng ta không được thông suốt,
nhưng thực tế là chúng ta thông minh hơn những gì mình tưởng.
Cuốn sách này sẽ giúp bạn:
• Làm thế nào để tư duy thông minh hơn
• Làm thế nào để tin tưởng vào trí tuệ của chính mình
Từng bước một, tôi sẽ chỉ cho các bạn thấy đâu là những nơi có
thể cải thiện khả năng nhận thức và quý trọng bản thân mình.
Cùng với sự giúp đỡ của những nhà tư tưởng, nhà tâm lý học xuất
sắc nhất, những con người thành công nhất trên thế giới, bạn sẽ
tìm thấy câu trả lời và các ví dụ để có thể:
• Tìm ra “tài năng xuất chúng” của chính mình
• Từ một người nghiệp dư trở thành chuyên gia
• Khám phá chiều sâu của tư duy sáng tạo
• Hiểu được bản chất tự nhiên của sự phán xét để đánh gi

In [30]:
import os
import fitz  # PyMuPDF

class Document:
    def __init__(self, doc_id, text, file_name):
        self.doc_id = doc_id
        self.text = text
        self.metadata = {'file_name': file_name}

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text()
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return text

def process_pdf_files_in_directory(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            text = extract_text_from_pdf(pdf_path)
            doc_id = os.path.splitext(filename)[0]  # Use filename as document ID
            document = Document(doc_id, text, filename)
            documents.append(document)
    return documents

def set_file_name_as_source(documents):
    for document in documents:
        document.metadata['file_name'] = document.doc_id

# Directory containing PDF files
pdf_directory = "../data"

# Process PDF files in the directory
documents = process_pdf_files_in_directory(pdf_directory)

set_file_name_as_source(documents)

# Print the converted documents with metadata
for doc in documents:
    print("Document ID:", doc.doc_id)
    print("Text:", doc.text[:100])  # Print only the first 100 characters of the text
    print("Metadata:", doc.metadata)
    print()

Document ID: Tu_duy_phan_bien
Text: GIỚI THIỆU
Cuốn sách này không viết về những suy nghĩ của riêng tôi,
nhưng tôi được thôi thúc bởi nh
Metadata: {'file_name': 'Tu_duy_phan_bien'}



In [43]:
for document in documents:
    document.metadata['file_name'] = document.metadata.get('source', 'Unknown')

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

# generator with openai models
generator = TestsetGenerator.with_openai()

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

In [75]:
import os

# TODO REMOVE ENVIRONMENT VARIABLES!!!
# get keys for your project from https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-4286bd95-4977-4515-b977-4ed1b1161187"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-910cf34d-2573-48be-8fa0-399e547c7c58"

# your openai key
# os.environ["OPENAI_API_KEY"] = ""

In [76]:
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")["baseline"]
fiqa_eval

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [77]:
# import metrics
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from ragas.metrics.critique import SUPPORTED_ASPECTS, harmfulness

# metrics you chose
metrics = [faithfulness, answer_relevancy, context_precision, harmfulness]

for m in metrics:
    print(m.name)
    # also init the metrics
    m.init_model()

faithfulness
answer_relevancy
context_precision
harmfulness


In [78]:
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")["baseline"]
fiqa_eval

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [79]:
row = fiqa_eval[0]
row["question"], row["answer"]

('How to deposit a cheque issued to an associate in my business into my business account?',
 '\nThe best way to deposit a cheque issued to an associate in your business into your business account is to open a business account with the bank. You will need a state-issued "dba" certificate from the county clerk\'s office as well as an Employer ID Number (EIN) issued by the IRS. Once you have opened the business account, you can have the associate sign the back of the cheque and deposit it into the business account.')

In [80]:
!python -m pip install langfuse

In [81]:
from langfuse import Langfuse

langfuse = Langfuse()

In [84]:
def score_with_ragas(query, chunks, answer):
    scores = {}
    for m in metrics:
        print(f"calculating {m.name}")
        scores[m.name] = m.score_single(
            {"question": query, "contexts": chunks, "answer": answer}
        )
    return scores